In [5]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [6]:
import time
import json

In [7]:
def every_downloads_chrome(driver):
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
        
    return driver.execute_script("""
        var items = downloads.Manager.get().items_;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl);
        """)

In [12]:
chrome_options = webdriver.ChromeOptions()
preference = {'download.default_directory': '/home/reyrizki/Downloads/audios', "safebrowsing.enabled": "false"}
chrome_options.add_experimental_option('prefs', preference)

In [37]:
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.maximize_window()

/home/reyrizki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [38]:
with open("source.json", "r") as source:
    videos = json.load(source);

    for video in videos:
        try:
            driver.get("https://ytmp3.cc/en13/")
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "input")))
            driver.find_element_by_id("input").send_keys(video['url'])
            driver.find_element_by_id("submit").click()
            
        except TimeoutException:
            driver.refresh()

        try:
            downloadButton = "//*[@id='buttons']/a[1]"
            time.sleep(30)
            driver.find_element_by_xpath(downloadButton).click()
        except TimeoutException:
            driver.refresh()

        while (len(driver.window_handles) > 1):
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
        
        time.sleep(10)

        WebDriverWait(driver, 120, 1).until(every_downloads_chrome)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.117)


In [ ]:
driver.quit()